<a href="https://colab.research.google.com/github/anshupandey/Natural_language_Processing/blob/master/Transfer_Learning_NLP_BERT_L2_H512_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 5.0 MB/s 
     |████████████████████████████████| 511.7 MB 6.1 kB/s 
     |████████████████████████████████| 5.8 MB 52.9 MB/s 
     |████████████████████████████████| 438 kB 72.4 MB/s 
     |████████████████████████████████| 1.6 MB 25.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninst

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import models,layers
import tensorflow_text as text

In [3]:
text_input = layers.Input(shape=(),dtype=tf.string)
prep = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = prep(text_input)

encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/2",trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"] #[batchsize,512]
sequence_ouptut = outputs["sequence_output"] #[batchsize,seqlength,512]

In [4]:
embedding_model = tf.keras.Model(text_input, pooled_output)
sentences = tf.constant(["(your text here)"])
print(embedding_model(sentences).shape)

(1, 512)


In [5]:
embedding_model2 = tf.keras.Model(text_input, sequence_ouptut)
sentences = tf.constant(["My name is anshu here, who are you"])
print(embedding_model2(sentences).shape)

(1, 128, 512)


In [6]:
sentences.shape

TensorShape([1])

In [7]:
doc1 = ["India"]
doc2 = ["I love coding"]
doc3 = ["I love python porgamming and Data Science. Machine Learning is amazing."]
data = [doc1,doc2,doc3]
for doc in data:
  doc = tf.constant(doc)
  print(embedding_model(doc).shape)

(1, 512)
(1, 512)
(1, 512)


In [8]:
for doc in data:
  doc = tf.constant(doc)
  print(embedding_model2(doc).shape)

(1, 128, 512)
(1, 128, 512)
(1, 128, 512)


# Text Classification

In [9]:
# Text Classification

In [10]:
!wget -q https://www.dropbox.com/s/p1z32pkhs9j7cv3/test_data.txt
!wget -q https://www.dropbox.com/s/ed3qcu1231e8ubx/train_5500.txt

In [11]:
train_data = open("train_5500.txt").readlines()
test_data = open("test_data.txt").readlines()

In [12]:
len(train_data)

5452

In [13]:
len(test_data)

500

In [14]:
train_data[:5]

['DESC:manner How did serfdom develop in and then leave Russia ?\n',
 'ENTY:cremat What films featured the character Popeye Doyle ?\n',
 "DESC:manner How can I find a list of celebrities ' real names ?\n",
 'ENTY:animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?\n',
 'ABBR:exp What is the full form of .com ?\n']

In [15]:
def prepare_data(data):
  x = []
  y = []
  for doc in data:
    docs = doc.split(" ")
    y.append(docs[0].split(":")[0])
    x.append(" ".join(docs[1:]).strip())
  return x,y


In [16]:
xtrain,ytrain = prepare_data(train_data)
xtest,ytest = prepare_data(test_data)

In [17]:
import pandas as pd

In [18]:
ytrain = pd.DataFrame(ytrain)
ytrain = pd.get_dummies(ytrain)
ytrain.head()

,0_ABBR,0_DESC,0_ENTY,0_HUM,0_LOC,0_NUM
0,0,1,0,0,0,0
1,0,0,1,0,0,0
2,0,1,0,0,0,0
3,0,0,1,0,0,0
4,1,0,0,0,0,0


In [19]:
ytest = pd.DataFrame(ytest)
ytest = pd.get_dummies(ytest)
ytest.head()

,0_ABBR,0_DESC,0_ENTY,0_HUM,0_LOC,0_NUM
0,0,0,0,0,0,1
1,0,0,0,0,1,0
2,0,0,0,1,0,0
3,0,1,0,0,0,0
4,0,0,0,0,0,1


In [20]:
categories = list(ytrain.columns)
categories = [i[2:] for i in categories]
categories

['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']

### Text Classification using Transfer learning

In [21]:
# create a function which will accept a text doc and return the corresponding vector
def sent2vec(doc):
  vector = embedding_model(tf.squeeze(tf.cast(doc,tf.string),axis=1))
  return vector

In [22]:
from tensorflow.keras import models,layers

In [23]:
input_layer = layers.Input(shape=(1,),dtype='string')
embedding_layer = layers.Lambda(sent2vec,output_shape=(512,))(input_layer)
dense1 = layers.Dense(256,activation='relu')(embedding_layer)
output_layer = layers.Dense(6,activation='softmax')(dense1)

model = models.Model(inputs=[input_layer],outputs=output_layer)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'pooler_transform/bias:0' shape=(512,) dtype=float32>
  <tf.Variable 'pooler_transform/kernel:0' shape=(512, 512) dtype=float32>
  <tf.Variable 'transformer/layer_3/output_layer_norm/beta:0' shape=(512,) dtype=float32>
  <tf.Variable 'transformer/layer_3/output_layer_norm/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'transformer/layer_3/output/bias:0' shape=(512,) dtype=float32>
  <tf.Variable 'transformer/layer_3/output/kernel:0' shape=(2048, 512) dtype=float32>
  <tf.Variable 'transformer/layer_3/intermediate/bias:0' shape=(2048,) dtype=float32>
  <tf.Variable 'transformer/layer_3/intermediate/kernel:0' shape=(512, 2048) dtype=float32>
  <tf.Variable 'transformer/layer_3/self_attention_layer_norm/beta:0' shape=(512,) dtype=float32>
  <tf.Variable 'transformer/layer_3/self_attention_layer_norm/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'transform

In [24]:
import numpy as np
xtrain = np.array(xtrain).reshape(-1,1)
xtest = np.array(xtest).reshape(-1,1)
print(xtrain.shape)
print(xtest.shape)

(5452, 1)
(500, 1)


In [30]:
model.fit(xtrain,ytrain,epochs=20,batch_size=64,validation_data=(xtest,ytest))

Epoch 1/20
86/86 [==============================] - 24s 272ms/step - loss: 0.9008 - accuracy: 0.6596 - val_loss: 0.5687 - val_accuracy: 0.8000
Epoch 2/20
86/86 [==============================] - 23s 266ms/step - loss: 0.8732 - accuracy: 0.6695 - val_loss: 0.6050 - val_accuracy: 0.7980
Epoch 3/20
86/86 [==============================] - 23s 267ms/step - loss: 0.8533 - accuracy: 0.6746 - val_loss: 0.5243 - val_accuracy: 0.8380
Epoch 4/20
86/86 [==============================] - 23s 267ms/step - loss: 0.8381 - accuracy: 0.6799 - val_loss: 0.5224 - val_accuracy: 0.8160
Epoch 5/20
86/86 [==============================] - 23s 267ms/step - loss: 0.8382 - accuracy: 0.6834 - val_loss: 0.5412 - val_accuracy: 0.8000
Epoch 6/20
86/86 [==============================] - 23s 267ms/step - loss: 0.8013 - accuracy: 0.6908 - val_loss: 0.4682 - val_accuracy: 0.8380
Epoch 7/20
86/86 [==============================] - 23s 269ms/step - loss: 0.7902 - accuracy: 0.6959 - val_loss: 0.5693 - val_accuracy: 0.8000

In [31]:
new_doc = ["Who is president of United States of America?","What is price for Harley Davidson?"]
new_doc = np.array(new_doc).reshape(-1,1)
output = model.predict(new_doc)

1/1 [==============================] - 0s 22ms/step


In [32]:
output2 = [np.argmax(i) for i in output]
output2 = [categories[i] for i in output2]
output2

['HUM', 'DESC']